# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
import sqlite3
from sqlalchemy import create_engine
import re

# Natural Langauge Toolkit
from nltk.corpus import stopwords
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import sys
import os
import re
from sqlalchemy import create_engine
import pickle

from scipy.stats import gmean
# import relevant modules from the sklearn
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin

# tokenize imports
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

In [ ]:
# load data from database
engine = create_engine('sqlite:///final_response.db')
df = pd.read_sql_table('final_responsetable', 'sqlite:///final_response.db')  

In [ ]:
df.describe() #not from the describe, child_alone has a whole column of zero, we should be able to just drop this.

From above and also below, only the 'related' may have a mistake, i.e. 0, 1, 2

In [4]:
# df['related'].value_counts()
df.groupby('related').count() # when using goupby, follow up with some metric, in this case count but could be mean - for example.

,id,message,original,genre,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,6122,6122,3395,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122
1.0,19906,19906,6643,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906
2.0,188,188,132,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188,188


In [5]:
def replace_with_majority(row):
    # import pdb; pdb.set_trace()
    if row == 2.0:
        return 1.0
    return row

df['related'] = df['related'].apply(replace_with_majority)
df['related'].value_counts() # great, we've gotten rid of the 2.0 which was most probably an error.

1.0    20094
0.0     6122
Name: related, dtype: int64

In [6]:
# ------------------------------------
X = df['message']
y = df.iloc[:,4:]
# ------------------------------------
features = df['message']
labels = df.iloc[:,4:] # take from 4 col, i.e. 'id', 'origignal', 'message', and 'genre' cols do not have any value and not labels.
# features # note, we can do some clearning here.

## Each column target is binary?
* loop through each column to determine if it really is 0 and 1

In [7]:
type_of_aid = y.columns
for col in type_of_aid:
    print(f'Aid: {col}')
    print(df[col].value_counts())

Aid: related
1.0    20094
0.0     6122
Name: related, dtype: int64
Aid: request
0.0    21742
1.0     4474
Name: request, dtype: int64
Aid: offer
0.0    26098
1.0      118
Name: offer, dtype: int64
Aid: aid_related
0.0    15356
1.0    10860
Name: aid_related, dtype: int64
Aid: medical_help
0.0    24132
1.0     2084
Name: medical_help, dtype: int64
Aid: medical_products
0.0    24903
1.0     1313
Name: medical_products, dtype: int64
Aid: search_and_rescue
0.0    25492
1.0      724
Name: search_and_rescue, dtype: int64
Aid: security
0.0    25745
1.0      471
Name: security, dtype: int64
Aid: military
0.0    25356
1.0      860
Name: military, dtype: int64
Aid: child_alone
0.0    26216
Name: child_alone, dtype: int64
Aid: water
0.0    24544
1.0     1672
Name: water, dtype: int64
Aid: food
0.0    23293
1.0     2923
Name: food, dtype: int64
Aid: shelter
0.0    23902
1.0     2314
Name: shelter, dtype: int64
Aid: clothing
0.0    25811
1.0      405
Name: clothing, dtype: int64
Aid: money
0.0    2

In [8]:
# -------------------------------------------------------------------------

In [9]:
def tokenize(text,url_place_holder_string="urlplaceholder"):
    """
    Tokenize the text function
    
    Arguments:
        text -> Text message which needs to be tokenized
    Output:
        clean_tokens -> List of tokens extracted from the provided text
    """
    
    # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    
    # Replace url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)

    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    
    #Lemmanitizer to remove inflectional and derivationally related forms of a word
    lemmatizer = nltk.WordNetLemmatizer()

    # List of clean tokens
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    return clean_tokens

In [10]:
pipeline1 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
            
        ])),

        ('classifier', MultiOutputClassifier(RandomForestClassifier()))
    ])

In [13]:

X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = pipeline1.fit(X_train, y_train)

In [8]:
# custom function for you to visualize all the cleaning of the text.
def normalize_clean(text):
    text = text.lower() # normalize
    text = re.sub(r"[^a-zA-Z0-9]", " ", text) # remove punctuations
    text = re.sub(r'http\S+', '', text) # remove urls
    text = re.sub(r'[%$#]', '', text) # remove special chars
    text = re.sub("[^a-zA-Z]", " ", text) # clean numbers 
    return text

features = features.apply(normalize_clean)
features

0        weather update   a cold front from cuba that c...
1                  is the hurricane over or is it not over
2                          looking for someone but no name
3        un reports leogane       destroyed  only hospi...
4        says  west side of haiti  rest of the country ...
5                   information about the national palace 
6                           storm at sacred heart of jesus
7        please  we need tents and water  we are in sil...
8          i would like to receive the messages  thank you
9        i am in croix des bouquets  we have health iss...
10       there s nothing to eat and water  we starving ...
11       i am in petionville  i need more information r...
12       i am in thomassin number     in the area named...
13       let s do it together  need food in delma     i...
14       more information on the      number in order f...
15       a comitee in delmas     rue   street   janvier...
16       we need food and water in klecin     we are dy.

Next we will remove stop words:
Stop words usually refers to the most commonly used words in a language (such as “the”, “a”, “an”, “in”)
This step is a one of the important steps to come up with more robust input. We will do all this in the tockenize def.

### 2. Write a tokenization function to process your text data

In [16]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def tokenize_custom(text):    
    # Extract the word tokens from the provided text row 
    tokens = nltk.word_tokenize(text)
    tokens = [w for w in tokens if not w in stop_words] #stopwords removal
    
    #Lemmanitizer to remove inflectional and derivationally related forms of a word
    lemmatizer = nltk.WordNetLemmatizer()
    cleaned_tokens = [lemmatizer.lemmatize(w).strip() for w in tokens]
    
    return cleaned_tokens

features = features.apply(tokenize_custom)
features

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


0        [Weather, update, -, cold, front, Cuba, could,...
1                                          [Is, Hurricane]
2                                 [Looking, someone, name]
3        [UN, report, Leogane, 80-90, destroyed, ., Onl...
4        [say, :, west, side, Haiti, ,, rest, country, ...
5                         [Information, National, Palace-]
6                            [Storm, sacred, heart, jesus]
7        [Please, ,, need, tent, water, ., We, Silo, ,,...
8             [I, would, like, receive, message, ,, thank]
9        [I, Croix-des-Bouquets, ., We, health, issue, ...
10       [There, 's, nothing, eat, water, ,, starving, ...
11       [I, Petionville, ., I, need, information, rega...
12       [I, Thomassin, number, 32, ,, area, named, Pyr...
13       [Let, 's, together, ,, need, food, Delma, 75, ...
14       [More, information, 4636, number, order, parti...
15       [A, Comitee, Delmas, 19, ,, Rue, (, street, ),...
16       [We, need, food, water, Klecin, 12, ., We, dyi.

In [11]:
def tokenize(text):    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


## Summary of text preprocessing
* Normalize - to lowercase and cleaned up punctuation
* tokenize - to each individual word
* removal of stop words - reduce the dimensionality we are working with.
* stemming/lemmetization - to further reduce words we work with.

In [12]:
# from lecture - here is a custom transformer for us to include in our pipeline
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence)) # using our defined token() above after pos.
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
# We will try various models and use GridSearchCV to get the est hyperparameters for each model.
models = [
#         {
#             "name": "LogisticRegression",
#             "estimator": LogisticRegression(verbose=True),
#             "hyperparameters":
#                 {
#                     "classifier__estimator__solver": ["newton-cg", "lbfgs", "liblinear"]
#                 }
#         },
#         {
#             "name": "KNeighborsClassifier",
#             "estimator": KNeighborsClassifier(),
#             "hyperparameters":
#                 {
#                     "classifier__estimator__n_neighbors": range(1,20,2),
#                     "classifier__estimator__weights": ["distance", "uniform"],
#                     "classifier__estimator__algorithm": ["ball_tree", "kd_tree", "brute"],
#                     "classifier__estimator__p": [1,2]
#                 }
#         },
        {
            "name": "RandomForestClassifier",
            "estimator": RandomForestClassifier(random_state=1, verbose=True),
            "hyperparameters":
                {
                    "classifier__estimator__n_estimators": [20, 60, 90],
                    "classifier__estimator__criterion": ["entropy", "gini"],
                    "classifier__estimator__max_depth": [2, 5, 10],
                    "classifier__estimator__max_features": ["log2", "sqrt"],
                    "classifier__estimator__max_features": [1, 5, 8],
                    "classifier__estimator__min_samples_split": [2, 3, 5]

                }
        }
    ]

In [14]:
# from instructions above, suggesting for us to use MultiOuptuClassifier
# strategy based on fitting one classifier per target.

def build_model(models):
    grids = []
    # create a pipeline + GridSearch obj for each model.
    for model in models:
        # import pdb; pdb.set_trace()
        pipeline = Pipeline([
            ('features', FeatureUnion([

                ('text_pipeline', Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()) # adding addtional features.
                ])),

                ('starting_verb', StartingVerbExtractor())
            ])),

            ('classifier', MultiOutputClassifier(model['estimator']))
        ])

        cv = GridSearchCV(pipeline, param_grid=model['hyperparameters'], scoring='f1_micro', n_jobs=-1)
        grids.append(cv)
    return grids

def display_results(cv, y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)
    print("\nBest Parameters:", cv.best_params_)

From above, we have CountVectorizer and tfidf running in parallel.
* convert to float/int for model to digest
* add additional features, i.e. tfidf transformation.

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.25, 
                                                    random_state=42)


In [ ]:
model_cv = build_model(models)
for model in model_cv:
    # import pdb; pdb.set_trace()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    display_results(model, y_test, y_pred)
    print("=" * 50)

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
y_inf_train = pipeline_trained.predict(X_train)
y_inf_test = pipeline_trained.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_inf_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.81      0.95      0.87      6598
               request       0.78      0.50      0.61      1472
                 offer       0.12      0.05      0.07        38
           aid_related       0.74      0.61      0.67      3545
          medical_help       0.57      0.25      0.35       701
      medical_products       0.60      0.30      0.40       446
     search_and_rescue       0.57      0.17      0.26       226
              security       0.17      0.03      0.04       160
              military       0.56      0.30      0.39       267
           child_alone       0.00      0.00      0.00         0
                 water       0.74      0.65      0.69       543
                  food       0.81      0.66      0.73       965
               shelter       0.76      0.53      0.63       775
              clothing       0.70      0.41      0.52       127
                 money       0.47      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.